In [1]:
import os
import requests
import json
from io import BytesIO
from PIL import Image
import base64
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
OPENROUTER_API_KEY = os.environ["OPENROUTER_API_KEY"]

In [19]:
with open("sample.txt", "r") as f:
    sample = f.read()

In [7]:
sample[:100]

'Summary of the article and thread (brief)\nArticle: OpenAI announces itself (Dec 2015) as a non‑profi'

In [33]:
prompt = """
    You are a professional slide maker.
    You are given texts and you need to generate a slide.
    1. figure out the main topic of the texts.
    2. decompose the texts into with their semantic meaning.
    3. generate slide images for each semantic meaning.
    Note: A slide has only one sub topic.
    Texts: {texts}
   
    """

In [21]:

def generate(prompt: str) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "google/gemini-2.5-flash-image-preview",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "modalities": ["image", "text"],
        "image_config": {
            "aspect_ratio": "16:9"
        }
    }

    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    if result.get("choices"):
        message = result["choices"][0]["message"]
        if message.get("images"):
            for idx, image in enumerate(message["images"]):
                image_url = image["image_url"]["url"]
                print(f"Generated image: {image_url[:50]}...")
   
                image_data = image_url.split(",", 1)[1]
                image_data = base64.b64decode(image_data)
                with BytesIO(image_data) as image_buffer:
                    image = Image.open(image_buffer)
                    image.save(f"output_{idx}.png", format="PNG")
    return result



In [3]:
def generate_text(prompt: str) -> dict:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "google/gemini-2.5-flash",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]      
    }

    response = requests.post(url, headers=headers, json=payload)
    result = response.json()
    return result

In [4]:
from langgraph.graph import StateGraph, START, END, add_messages

In [5]:
from typing import Annotated, TypedDict
from langchain.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage

In [6]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    prompt: str

In [22]:
PLANNER_PROMPT = """
    You are a planner to make slides.
    You are given texts and you need to plan for making slides.
    1. figure out the main topic of the texts.
    2. decompose the texts into with their semantic meaning.
    3. make plans to generate slide images for each semantic meaning.
    Note: A slide has only one sub topic.
    Texts: {texts}
   
    """

In [65]:
res = generate(PLANNER_PROMPT.format(texts=sample))

In [66]:
res["choices"][0]["message"]

{'role': 'assistant',
 'content': 'Here is a plan for creating slides based on the provided text.\n\n### 1. Main Topic\n\n**The Evolution of OpenAI: A Retrospective on the 2015 Vision vs. The 2024 Reality**\n\n---\n\n### 2. & 3. Semantic Decomposition and Image Slide Plans\n\nHere is the breakdown of the text into semantic sub-topics, along with a plan for the visual design of each slide.\n\n**Slide 1: Title Slide**\n*   **Semantic Meaning:** Introduction to the presentation topic.\n*   **Image Plan:** A split-screen image. On the left, a vintage 2015-style computer terminal displaying "OpenAI: Non-Profit Research Lab." On the right, a futuristic 2024 interface displaying "OpenAI: Commercial AI Giant." A timeline arrow connects them. Title text overlaid: "OpenAI: The 2015 Vision vs. The 2024 Reality."\n\n**Slide 2: The Genesis (2015)**\n*   **Semantic Meaning:** OpenAI\'s original launch announcement, emphasizing its non-profit status, mission for humanity, freedom from financial const

In [16]:
def planner(state: GraphState) -> GraphState:
    messages = state["messages"][-1].content
    res = generate(PLANNER_PROMPT.format(texts=messages))
    return {"messages": [AIMessage(content=res["choices"][0]["message"]["content"])]}

def generate_slide(state: GraphState) -> GraphState:
    request = state["messages"][-1].content
    res = generate(request)
    return {"messages": [AIMessage(content=res["choices"][0]["message"]["content"])]}

def test_generate(state: GraphState) -> GraphState:
    print(state["messages"][-1].content)
    request = state["messages"][-1].content
    res = generate_text(request)
    return {"messages": [AIMessage(content=res["choices"][0]["message"]["content"])]}


In [17]:
graph = StateGraph(GraphState)
graph.add_node("planner", planner)
graph.add_node("generate_slide", generate_slide)
graph.add_edge(START, "planner")
graph.add_edge("planner", "generate_slide")
graph.add_edge("generate_slide", END)

In [18]:
app = graph.compile()

In [23]:
for chunk in app.stream({"messages": [{"role": "user", "content": sample}]}, stream_mode="values"):
    print(chunk)


{'messages': [HumanMessage(content="Summary of the article and thread (brief)\nArticle: OpenAI announces itself (Dec 2015) as a non‑profit AI research lab, “unconstrained by a need to generate financial return.” Mission: advance digital intelligence to benefit humanity; focus on openness (papers, code, maybe patents), and safety (“AI should be an extension of individual human wills… broadly and evenly distributed”). It’s explicitly deep‑learning‑oriented, with a star founding team (Sutskever, Karpathy, Kingma, Schulman, etc.), $1B in committed funding from Musk, Altman, Thiel, AWS, Infosys, YC Research, etc., and rhetoric about long‑term AI risks and benefits.\n\nThread themes:\n\nExcitement at the team and funding, plus jokes about the casual mention of $1B.\nArguments over:\nWhether deep learning / neural nets can lead to AGI or will plateau.\nWhether AI risk talk is “fear‑mongering” or a prudent early start.\nWhether OpenAI is really about safety vs just joining the deep‑learning ar